## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Pasvalys,LT,56.06,24.40,67.32,96,64,5.82,broken clouds
1,1,Atuona,PF,-9.80,-139.03,77.83,71,1,17.92,clear sky
2,2,Ribeira Grande,PT,38.52,-28.70,72.01,85,55,8.61,broken clouds
3,3,Talnakh,RU,69.49,88.40,61.65,86,96,4.45,light rain
4,4,New Norfolk,AU,-42.78,147.06,55.92,63,53,5.01,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your vacation? "))
max_temp = float(input("What is the maximum temperature you would like for your vacation? "))

What is the minimum temperature you would like for your vacation? 60
What is the maximum temperature you would like for your vacation? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Pasvalys,LT,56.06,24.40,67.32,96,64,5.82,broken clouds
1,1,Atuona,PF,-9.80,-139.03,77.83,71,1,17.92,clear sky
2,2,Ribeira Grande,PT,38.52,-28.70,72.01,85,55,8.61,broken clouds
3,3,Talnakh,RU,69.49,88.40,61.65,86,96,4.45,light rain
5,5,Avarua,CK,-21.21,-159.78,77.05,65,21,4.61,few clouds
6,6,Teguldet,RU,57.31,88.17,61.92,63,92,2.37,overcast clouds
7,7,Lieksa,FI,63.32,30.02,60.94,89,51,3.74,broken clouds
8,8,Ponta Do Sol,PT,32.67,-17.10,70.72,77,3,3.98,clear sky
10,10,Yellowknife,CA,62.46,-114.35,61.29,48,57,8.99,broken clouds
11,11,Hamilton,US,39.18,-84.53,76.35,82,1,0.00,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.notnull().sum()

City_ID                490
City                   490
Country                490
Lat                    490
Lng                    490
Max Temp               490
Humidity               490
Cloudiness             490
Wind Speed             490
Current Description    490
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Pasvalys,LT,67.32,broken clouds,56.06,24.40,
1,Atuona,PF,77.83,clear sky,-9.80,-139.03,
2,Ribeira Grande,PT,72.01,broken clouds,38.52,-28.70,
3,Talnakh,RU,61.65,light rain,69.49,88.40,
5,Avarua,CK,77.05,few clouds,-21.21,-159.78,
6,Teguldet,RU,61.92,overcast clouds,57.31,88.17,
7,Lieksa,FI,60.94,broken clouds,63.32,30.02,
8,Ponta Do Sol,PT,70.72,clear sky,32.67,-17.10,
10,Yellowknife,CA,61.29,broken clouds,62.46,-114.35,
11,Hamilton,US,76.35,clear sky,39.18,-84.53,


In [ ]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.") 

In [ ]:
hotel_df.notnull().sum()

In [ ]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()

In [ ]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig